In [ ]:
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
from pylissom.utils.training import Pipeline
from torch.utils.data import Dataset, DataLoader
from pylissom.datasets.datasets import OrientatedGaussians, ThreeDotFaces

In [ ]:
from pylissom.models.models import get_lissom 
from pylissom.utils.plotting import *
from pylissom.utils.helpers import debug

###### If this takes more than 5 seconds,  it's a problem with the pytorch and cuda versions, only happens first time cuda is called
###### If this takes more than 30 seconds, maybe your cuda is broken

In [ ]:
import torch
foo = torch.Tensor(2)
%time foo.cuda()

## Training variables

In [ ]:
size = 24
count = 10000
epochs = 1
cuda = False
gaussians_per_image = 2

## Init and train Lissom

In [ ]:
lissom, optimizer, _ = get_lissom(retinal_density=size, lgn_params='lgn', rlissom_params='rlissom', optim_params='optim')
display(lissom)

In [ ]:
gaussians_inputs = OrientatedGaussians(size=size, length=count, gaussians=gaussians_per_image)
# gaussians_inputs.cuda()
gaussians_inputs = ThreeDotFaces(size=size, length=count, faces=1)
train_loader = DataLoader(gaussians_inputs, shuffle=True, pin_memory=False)

In [ ]:
sample, test_sample = iter(train_loader).next()

In [ ]:
plot_tensor(sample, shape=(size, size))

In [ ]:
pipe = Pipeline(lissom, optimizer, cuda=cuda, log_interval=10)

## Inpect activation

In [ ]:
gauss, _ = iter(train_loader).next()
plot_tensor(gauss, (size, size), vmin=-1, vmax=1)

In [ ]:
from pylissom.nn.modules.lissom import *
from pylissom.nn.modules import register_recursive_forward_hook, input_output_hook

register_recursive_forward_hook(lissom, input_output_hook)
inp = Pipeline.process_input(torch.autograd.Variable(gauss))
out = lissom(inp)
plot_layer_activation(lissom, 'lissom')

## Train

In [ ]:
pipe.train(train_loader, epoch=0)

## Calculate Orientations Maps

In [ ]:
from pylissom.utils.orientation_maps import *
orientations = 180

keys_arrays = get_oriented_lines(size, orientations=orientations)

In [ ]:
keys_arrays = {k: map(lambda l: torch.autograd.Variable(l.cuda()), lines) for k, lines in keys_arrays.items()}

In [ ]:
om = OrientationMap(model=lissom, inputs=keys_arrays)

orientation_map = om.get_orientation_map()
orientation_hist = om.get_orientation_hist()

In [ ]:
plot_orientation_map(orientation_map)
plt.colorbar()
plt.show()

In [ ]:
plot_orientation_hist(orientation_hist)
plt.show()

mean, std = metrics_orientation_hist(orientation_hist)
print("Promedio: %{:.2f}".format(mean*100))
print("Standard deviation: %{:.2f}".format(std*100))